In [7]:
import sqlite3
import json 
import os
import pandas as pd 

In [8]:
import requests
from urllib.parse import urlparse

In [9]:
import fastwer

In [34]:
from jiwer import wer

In [3]:
!pip install pandas

     |████████████████████████████████| 9.7MB 922kB/s eta 0:00:01
     |████████████████████████████████| 512kB 20.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [27]:
!poetry add fastwer

Using version ^0.1.3 for fastwer

Updating dependencies
Resolving dependencies... (43.1s)Resolving dependencies... (19.5s)Resolving dependencies... (41.0s)

Writing lock file

Package operations: 2 installs, 0 updates, 0 removals

  • Installing pybind11 (2.6.2): Pending...
  • Installing pybind11 (2.6.2): Downloading... 0%
  • Installing pybind11 (2.6.2): Downloading... 70%
  • Installing pybind11 (2.6.2): Downloading... 100%
  • Installing pybind11 (2.6.2): Downloading... 100%
  • Installing pybind11 (2.6.2): Installing...
  • Installing pybind11 (2.6.2)
  • Installing fastwer (0.1.3): Pending...
  • Installing fastwer (0.1.3): Downloading... 0%
  • Installing fastwer (0.1.3): Downloading... 100%
  • Installing fastwer (0.1.3): Downloading... 100%
  • Installing fastwer (0.1.3): Installing...
  • Installing fastwer (0.1.3)


In [10]:

conn = sqlite3.connect("589.sqlite")
cursor = conn.cursor()
cursor.execute("select * from data") 
data_set= cursor.fetchall()


In [12]:
data_set[0]

(871481,
 '{"text": "my Tata Sky is not working I have balance in my account why", "uuid": "b0fe8be8-1814-4cfd-90d6-88773c5fc4f6", "state": "RECORD_SUBSEQUENT_QUERY", "filter": {"current_state": "RECORD_SUBSEQUENT_QUERY"}, "call_id": 2319504, "reftime": "2020-08-25T13:55:48Z", "audio_url": "https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F588fb072-f7b8-437c-a922-509cdcfc4de8%2Fb0fe8be8-1814-4cfd-90d6-88773c5fc4f6.flac", "prediction": {"graph": {"input": [[[{"confidence": 1, "transcript": "my Tata Sky is not working I have balance in my account why"}]]]}}, "alternatives": [[{"confidence": 1, "transcript": "my Tata Sky is not working I have balance in my account why"}]], "conversation_id": 23330549}',
 '{"text": "my Tata Sky is not working I have balance in my account why", "type": "TRANSCRIPT", "alt-index": 0}',
 0,
 '2021-03-01 10:35:05.254496+00:00')

In [41]:
"""
Script for testing out ASR server.

Usage:
  example_client.py mic [--n-secs=<n-secs>] [--model=<model>] [--lang=<lang>] [--sample-rate=<sample-rate>] [--max-alternatives=<max-alternatives>] [--stream] [--raw] [--pcm] [--word-level]
  example_client.py <file>... [--model=<model>] [--lang=<lang>] [--sample-rate=<sample-rate>] [--max-alternatives=<max-alternatives>] [--stream] [--raw] [--pcm] [--word-level]

Options:
  --n-secs=<n-secs>                         Number of seconds to record the audio for before making a request. [default: 5]
  --model=<model>                           Name of the model to hit. [default: general]
  --lang=<lang>                             Language code of the model. [default: en]
  --sample-rate=<sample-rate>               Sampling rate to use for the audio. [default: 8000]
  --max-alternatives=<max-alternatives>     Number of maximum alternatives to query from the server. [default: 10]
  --raw                                     Flag that specifies whether to stream raw audio bytes to server.
  --pcm                                     Flag that specifies whether to send raw pcm bytes.
  --word-level                              Flag to enable word level features from server.
"""
import time
import random
import threading
import traceback
from pprint import pprint
from typing import List

from docopt import docopt
from pydub import AudioSegment

from kaldi_serve import KaldiServeClient, RecognitionAudio, RecognitionConfig
from kaldi_serve.utils import (
    chunks_from_file,
    chunks_from_mic,
    raw_bytes_to_wav
)

ENCODING = RecognitionConfig.AudioEncoding.LINEAR16


def parse_response(response):
    output = []

    for res in response.results:
        output.append([
            {
                "transcript": alt.transcript,
                "confidence": alt.confidence,
                "am_score": alt.am_score,
                "lm_score": alt.lm_score,
                "words": [
                    {
                        "start_time": word.start_time,
                        "end_time": word.end_time,
                        "word": word.word,
                        "confidence": word.confidence
                    }
                    for word in alt.words
                ]
            }
            for alt in res.alternatives
        ])
    return output


def transcribe_chunks_streaming(client, audio_chunks, model: str, language_code: str,
                                sample_rate=8000, max_alternatives=10, raw: bool=False,
                                word_level: bool=False, chunk_size: float=0.5):
    """
    Transcribe the given audio chunks
    """

    response = {}

    try:
        if raw:
            config = lambda chunk_len: RecognitionConfig(
                sample_rate_hertz=sample_rate,
                encoding=ENCODING,
                language_code=language_code,
                max_alternatives=max_alternatives,
                model=model,
                raw=True,
                word_level=word_level,
                data_bytes=chunk_len
            )

            start = [None]
            def audio_params_gen(audio_chunks, start):
                for chunk in audio_chunks[:-1]:
                    yield config(len(chunk)), RecognitionAudio(content=chunk)
                    time.sleep(chunk_size)
                start[0] = time.time()
                yield config(len(audio_chunks[-1])), RecognitionAudio(content=audio_chunks[-1])

            response = client.streaming_recognize_raw(audio_params_gen(audio_chunks, start), uuid=str(random.randint(1000, 100000)))
            end = time.time()
            print(f"{((end - start[0])*1000):.2f}ms")
        else:
            audio = (RecognitionAudio(content=chunk) for chunk in audio_chunks)
            config = RecognitionConfig(
                sample_rate_hertz=sample_rate,
                encoding=ENCODING,
                language_code=language_code,
                max_alternatives=max_alternatives,
                model=model,
                word_level=word_level
            )
            response = client.streaming_recognize(config, audio, uuid=str(random.randint(1000, 100000)))
    except Exception as e:
        traceback.print_exc()
        print(f'error: {str(e)}')

    #pprint(parse_response(response))
    
    return parse_response(response)

def transcribe_chunks_bidi_streaming(client, audio_chunks, model: str, language_code: str,
                                     sample_rate=8000, max_alternatives=10, raw: bool=False,
                                     word_level: bool=False):
    """
    Transcribe the given audio chunks
    """
    response = {}

    try:
        if raw:
            config = lambda chunk_len: RecognitionConfig(
                sample_rate_hertz=sample_rate,
                encoding=ENCODING,
                language_code=language_code,
                max_alternatives=max_alternatives,
                model=model,
                raw=True,
                data_bytes=chunk_len,
                word_level=word_level,
            )

            def audio_params_gen(audio_chunks):
                for chunk in audio_chunks:
                    yield config(len(chunk)), RecognitionAudio(content=chunk)

            response_gen = client.bidi_streaming_recognize_raw(audio_params_gen(audio_chunks), uuid=str(random.randint(1000, 100000)))
        else:
            config = RecognitionConfig(
                sample_rate_hertz=sample_rate,
                encoding=ENCODING,
                language_code=language_code,
                max_alternatives=max_alternatives,
                model=model,
                word_level=word_level
            )

            def audio_chunks_gen(audio_chunks):
                for chunk in audio_chunks:
                    yield RecognitionAudio(content=chunk)

            response_gen = client.bidi_streaming_recognize(config, audio_chunks_gen(audio_chunks), uuid=str(random.randint(1000, 100000)))
    except Exception as e:
        traceback.print_exc()
        print(f'error: {str(e)}')
    
    output = []
    for response in response_gen:
        output.append(parse_response(response))
        pprint(parse_response(response))
    return output     


def decode_files(client, audio_paths: List[str], model: str, language_code: str,
                 sample_rate=8000, max_alternatives=10, raw: bool=False,
                 pcm: bool=False, word_level: bool=False, chunk_size: float=0.5):
    """
    Decode files using threaded requests
    """
    chunked_audios = [chunks_from_file(x, sample_rate=sample_rate, chunk_size=chunk_size, raw=raw, pcm=pcm) for x in audio_paths]
    
    output =[]
    for chunks in chunked_audios:
        output.append(transcribe_chunks_streaming(client, chunks, model, language_code,
                  sample_rate, max_alternatives, raw,
                  word_level, chunk_size))
        
    return output



'''if __name__ == "__main__":
    args = docopt(__doc__)
    client = KaldiServeClient()

    # args
    model = args["--model"]
    language_code = args["--lang"]
    sample_rate = int(args["--sample-rate"])
    max_alternatives = int(args["--max-alternatives"])

    # flags
    raw = args['--raw']
    pcm = args['--pcm']
    word_level = args["--word-level"]

    if args["mic"]:
        transcribe_chunks_bidi_streaming(client, chunks_from_mic(int(args["--n-secs"]), sample_rate, 1),
                                         model, language_code, sample_rate, max_alternatives,
                                         raw or pcm, word_level)
    else:'''

client = KaldiServeClient()
model = "tatasky-v1"
language_code = "en"
output_df = []

for i in range(len(data_set)):
    try:
        row_data = json.loads(data_set[i][1])
        tag_text = json.loads(data_set[i][2])["text"] 
        file_name = row_data["audio_url"].split("/")[-1].split(".")[0] + ".wav"
        VASR_transcription = decode_files(client,[f"tatasky_wav_audios/{file_name}"],model,language_code)
        GASR_transcription = row_data["prediction"]["graph"]["input"][0][0][0]["transcript"]
        print(GASR_transcription," ",VASR_transcription[0][0][0]['transcript'])
        output_df.append([file_name,tag_text,VASR_transcription[0][0][0]['transcript'],GASR_transcription])
    except KeyError:
        pass
    except IndexError:
        pass

output_df = pd.DataFrame(output_df,columns=["file_name","tag_text","VASR_transcription","GASR_transcription"])
output_df.to_csv("tatasky_report_v1.csv")

my Tata Sky is not working I have balance in my account why   <UNK> is not working have <UNK> why
I want Colours channel   i want the life <UNK>
I want Aastha channel   i want of the channel
I want Discovery channel   i want discovery channel
mujhe apna Jo connection hai mere bhai ko dena tha   <UNK> jo connection <UNK>
I want an Android box   i want an android box
change my slot timings   i am <UNK> must not coming
I want to see colours Kannada   i want to see colors kannada
Mera workout status kya hai   <UNK> work order status
Pandit Mohabbat Karun balance and recharge details   go my current balance recharge details
yah naya Android box kitne ka Padega   <UNK> box <UNK>
Tata Sky box is not working and somebody was supposed to come today but so far I am not received   my tata sky box is not working on somebody was supposed to come today but also for not received
Kya Mera Bhasha Badal sakte ho   <UNK>
remote replacement request was raised a week back still not delivered   remote repla

Mujhe Janna tha ki kitne ka recharge karun   <UNK> it may carry <UNK>
ok   book a
I want my remote is not working I want to change this   <UNK> remote is not working
my remote is not working   my remote is not working
what are the charges   <UNK> recharge
I want to know my business charge   i want to know my dish charge
queries related to Tata Sky binge   <UNK> query related to tata sky binge
refund   refund
refund   refund
I want a new schedule   wanted new a
Bhojpuri nautanki   no its still not working
I want to know the charges right now   i want to know the charges right now
yes   yes <UNK> the remote remote
I want to know about my remote   i want to know about my remote
when can I meet Rajkumar Rao   when can i meet <UNK>
thank you   thank you
number   <UNK>
now   now he
thank you bhai   why
yes   repeat act
yes I tried   yes i tried
how many days will it take   <UNK> how many days
non banking   <UNK> thanking
hello   <UNK> <UNK>
yes I want to know how to recharge the account   ye

can I do address change on my own and how do I do it   connect the address change on my own a <UNK>
I have two boxes I would do want to know what I give 11 box to my friend   hey i have two boxes i would do want to know why <UNK> my friend <UNK>
chemistry Tata Sky associate   tata sky associate
MI remote is not working I want a new remote   remote is not working i want new remote
actually I am on leave so I will not giving is there any way that I can know   actually <UNK> any leave so i will not giving is there anyway very can allow us
can you help me with my current stock on my in a primary box and other multi TV   can you help me with my current pack on my in primary box under the multi tv
I want to buy another Tata Sky box   <UNK> by on the tata sky box
remote replacement in confirm the status   <UNK> ideas for remote replacement in confirm the status
I want new channel   i want you channel
can I have a status update on my pending request   can i have citrus update on my pending req

fast schedule   passed should due
I want a second connection   i want a second connection
28 of October   twenty its of october
importance should be fine   evening four pm
how can he come in   ya can he come in the
yeah can I please reschedule it   ya can i please <UNK>
unknown person later   i am not at home <UNK> person later
now only   now
when will you an engineer come to resolve my shoe   come resolve my issue
I said no I'm not here for remote replacement   i felt below <UNK> not here for repairing remote replacement
6:00 p.m.   six pm
I want to change the box   i want to change the box
blue light on the Tata Sky setup box   no light on the tata sky setup box
I want to deactivate the connection now   <UNK> reactivate the connection now
I want to know my recharge status   i want to know my plan
no signal aa raha hai   no signal ah
I don't want any help thank you   i dont want any help thank you
Tata Sky binge channel   one connected been shared with an
pizza   id <UNK>
know my mont

sexy video   <UNK>
Narendra Modi   my remote
what are you think   what
I want to know about remote   i want to know about remote
I will get what are you saying   i did what
I want to go to place with   i want remote replacement
yasu   so your not scheduled after two hours
I want to talk to an agent   i want to talk to an agent
thank you bhai   twenty by
nothing   working
kya it's working now   ya its working now
it's working   its working
I have gone through some   i have gone through some
thank you   thank you
please go ahead   go ahead
yeah I think so   ya <UNK> things
yes actually I want to change the date   yes actually i want to change the date
can you please schedule for December 13th   can you please <UNK> for a december thirteenth
ok I know leave it thanks   okay on only with
yes I had changed   add change
yeah I tried undri   ya i tried already
please tell me about charges   but charges
bahut   <UNK>
yes yes I have already tried it   yes have already tried
I want to book a slo

In [29]:
output_df = pd.read_csv("tatasky_report_v1.csv")
output_df["VASR_first_alternative"] = output_df["VASR_transcription"].apply(lambda x:eval(x)[0][0][0]["transcript"])

In [40]:
gasr_d = list(output_df["GASR_transcription"].values)
vasr_d = list(output_df["VASR_first_alternative"].values)
gt = list(output_df["tag_text"].values)
print(len(gasr_d),len(vasr_d),len(gt))
#assert len(gasr_d) == len(vasr_d)
gasr_error = wer(gt, gasr_d)
vasr_error = wer(gt, vasr_d)
print("wer gasr: ", gasr_error)
print("wer vasr: ", vasr_error)

655 655 655
wer gasr:  0.2812789620018536
wer vasr:  0.46223354958294716


In [30]:
output_df.head()

,Unnamed: 0,file_name,tag_text,VASR_transcription,GASR_transcription,VASR_first_alternative
0,0,hermes-cca%2F2020-08-25%2F588fb072-f7b8-437c-a...,my Tata Sky is not working I have balance in m...,[[[{'transcript': 'my tata sky is not working ...,my Tata Sky is not working I have balance in m...,my tata sky is not working i ve maximum why
1,1,hermes-cca%2F2020-08-19%2F10476b73-dd66-43a2-9...,I want Colours channel,"[[[{'transcript': 'i want the less chairman', ...",I want Colours channel,i want the less chairman
2,2,hermes-cca%2F2020-08-19%2F10476b73-dd66-43a2-9...,I want Aastha channel,"[[[{'transcript': 'i want of the channel', 'co...",I want Aastha channel,i want of the channel
3,3,hermes-cca%2F2020-08-19%2F0058d87c-fedb-4076-b...,I want Discovery channel,"[[[{'transcript': 'i want discovery channel', ...",I want Discovery channel,i want discovery channel
4,4,hermes-cca%2F2020-09-04%2Fe1d8a99f-dfc4-4c41-8...,mujhe apna Jo connection hai mere bhai ko dena...,[[[{'transcript': 'mujhe naya connection hai m...,mujhe apna Jo connection hai mere bhai ko dena...,mujhe naya connection hai mera bhai kareena


In [27]:
output_df.head()

,Unnamed: 0,file_name,tag_text,VASR_transcription,GASR_transcription
0,0,hermes-cca%2F2020-08-25%2F588fb072-f7b8-437c-a...,my Tata Sky is not working I have balance in m...,[[[{'transcript': 'my tata sky is not working ...,my Tata Sky is not working I have balance in m...
1,1,hermes-cca%2F2020-08-19%2F10476b73-dd66-43a2-9...,I want Colours channel,"[[[{'transcript': 'i want the less chairman', ...",I want Colours channel
2,2,hermes-cca%2F2020-08-19%2F10476b73-dd66-43a2-9...,I want Aastha channel,"[[[{'transcript': 'i want of the channel', 'co...",I want Aastha channel
3,3,hermes-cca%2F2020-08-19%2F0058d87c-fedb-4076-b...,I want Discovery channel,"[[[{'transcript': 'i want discovery channel', ...",I want Discovery channel
4,4,hermes-cca%2F2020-09-04%2Fe1d8a99f-dfc4-4c41-8...,mujhe apna Jo connection hai mere bhai ko dena...,[[[{'transcript': 'mujhe naya connection hai m...,mujhe apna Jo connection hai mere bhai ko dena...


In [35]:
def audio_url_to_file(url,directory_name):
    file_name = url.split("/")[-1]
    if not os.path.exists(directory_name+"/"+file_name):
        r = requests.get(url, allow_redirects=True)
        path = urlparse(url).path
        #file_name = str(path[1:])
        save_path = directory_name+'/'+ file_name
        open(save_path, 'wb').write(r.content)
        return save_path
    return None

In [21]:
def flac2wav(flac_path):
    # check if flac:
    file_name, ext = os.path.splitext(flac_path)
    if ext == '.flac':
        command = "ffmpeg -i " + flac_path + " " + file_name + ".wav" 
        os.system(command)
        new_path = file_name + ".wav"
        return new_path
     return flac_path    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [36]:
for i in range(len(data_set)):
    row_data = json.loads(data_set[i][1])
    audio_url = row_data["audio_url"]
    print("url:",audio_url)
    audio_url_to_file(audio_url,"tatasky_audios")
    #if not os.path.exists(f"tatasky_audios/{file_name}"):
        #os.system(f"aws s3 cp s3://tog-call-data-generation/{file_name} tatasky_audios")

url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F588fb072-f7b8-437c-a922-509cdcfc4de8%2Fb0fe8be8-1814-4cfd-90d6-88773c5fc4f6.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-19%2F10476b73-dd66-43a2-9c72-ec029e1c87f6%2Fc24e13fa-220e-4269-9234-f0285cf8ad50.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-19%2F10476b73-dd66-43a2-9c72-ec029e1c87f6%2F796a934f-a9dd-40d6-8cd6-697821951662.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-19%2F0058d87c-fedb-4076-ba67-9cfc4cce7694%2Fc33ab1bd-6d02-44e2-b576-b60610768229.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-09-04%2Fe1d8a99f-dfc4-4c41-8ee1-f1c92250bd46%2F40621e5b-240f-4afa-bf93-c4338e5377cf.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-09-04%2F8f1ead26-5dc5-4e5e-887e-ab69aa21bca8%2F2cf136b9-622e-4c32-a71b-4f9a23a681c6.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-18%2Fced1c831-1a84-4195-96b9-b3204348e9ab%2F23f18f

url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2Fdc9d71be-f8d5-436d-9632-41eb9082d59c%2F7c0b4c4c-3f37-4936-8892-feb2bce0a342.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F1fe883b0-b4cb-43fb-ba23-7cee70143f19%2Fe0caef7d-d390-47aa-8f87-b59671ca5a8b.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F1fe883b0-b4cb-43fb-ba23-7cee70143f19%2F30050f43-cd81-49fb-9cde-38775708cf86.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F02d99193-af3a-4807-8851-155ee1162042%2F7a9548b9-c540-48b0-a8e1-03ad1fcfd634.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F02d99193-af3a-4807-8851-155ee1162042%2Fe44121b2-a7ed-4a4d-81a8-b5681c815020.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-24%2F4ff8abec-1f84-4839-baa1-a81aae5af8a1%2F5a554325-ce75-487a-abd3-90463d8c2736.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-18%2Fced1c831-1a84-4195-96b9-b3204348e9ab%2F0d5e9b

url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-10-07/46adcbc8-eb68-43a4-8a09-e98abeae3c81/7ac0837e-42ad-429d-b52c-372919a1450f.wav
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-10-29/b452cb25-2382-4002-b599-3caa7568e3b2/2dadcbb5-a5c3-400c-abf8-b324342883ad.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-25/2d860fe7-ff5f-401d-b149-428948df5df0/88b87568-3093-49fa-a661-2c795a3f2b01.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-27/9e0b5ac7-34a2-465e-91ac-460c90a913df/3fb67f97-7e17-400d-aefc-54c8eed3b600.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-01/0e663474-2613-4066-9bca-d5a03e767143/00dba92c-bd80-485c-a55e-527a59780953.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003GN%2Fdafe5252-86a6-44d6-bd37-d97a2fc5b221.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003GN%2F291a73f1-c9c3-4a47-8281-4e6c4b3a6ea9.flac
url

url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2021-01-13/019F3IH86OFULB386S3182LAES0000UQ/13e8e94e-a727-490c-8622-037013411494.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-11/7dd406a6-5b7f-47b2-ac2a-a47522359e78/5c2c5319-2c2a-47c0-9a56-3c904ad57077.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003GS%2F2191bfa6-4c1d-475d-b69d-65b069fced0f.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-04/cd1c85e2-3936-4ea8-b848-1bffd6e36cdc/f36c3667-a0eb-4ad1-b380-7e749a1666d6.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-27/dc743e60-b85e-41f2-9ca6-aae66eb991d1/3e2fa173-f327-4658-8f6d-d04dc53f9953.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-23/11526a20-bc76-4a76-895c-7f44c19277f4/d7c6bc3c-14e5-4612-8bf7-d48986d4f294.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-24/946b91a3-f204-4256-b098-a571e15b3add/1d7cac67-06f2-4838-ae3b-5ab1840881f6.flac
url: htt

url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2021-01-12/3c8624d64cdf4f16/87d29548-2375-4446-9fbf-5d794049a9a3.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2021-01-12/019F3IH86OFULB386S3182LAES0000M7/af902220-f10d-4c0c-9322-01cd536491e2.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-17/e531c532-cd01-4ae5-aa83-9c3fd3187128/9b933f8e-635a-4521-8740-4140185fac50.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-04/8214c50f-7692-42d1-b933-3ef5fa39a1a5/fe03186a-bb2a-4e93-99d1-bd12ad7734ce.wav
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-04/b3fa8acd-adaf-40d0-ad8e-93d8af65d2d0/708cb725-7593-411e-a64e-e224ea0d648a.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-04/5edcb689-5e39-4f07-8299-12f569c017da/2772281a-f82e-4923-a15c-1129d79985e9.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-16/283c4bf6-5a9d-466f-bea8-ecbb9176bba6/6d9a8ef2-86b8-4e3c-90c4-415c2c46bac6.flac
url: https://hermes-cca.s3.ap-s

...

url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-10-29/8a520671-3142-4a9f-b996-e9c6b8242a58/51e54d8b-44a4-4861-a837-eca9990e4925.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-10-29/c0968532-05ad-4958-a484-3cdc50572fc5/56f8dcbe-3ff1-4aa9-88bd-57e64a45d20b.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F0d8bf6e3-ae6c-4e29-aa90-215c89186300%2Ff3612b20-b817-4fa1-a891-787248e95cf0.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F0d8bf6e3-ae6c-4e29-aa90-215c89186300%2Fe9babe2f-e729-478f-a1e5-e8501f1b4cb6.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-19%2F0058d87c-fedb-4076-ba67-9cfc4cce7694%2F1781207c-c899-41c6-8a76-ab3fe4abdc5d.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-19%2F0058d87c-fedb-4076-ba67-9cfc4cce7694%2Fab0caf35-3c38-4a20-826a-c82693744e9e.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F08754335-e23d-4524-8399-a2644caf4c05%2F4f6b5614-d8aa-4ced

url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-17/190e912c-632f-465a-ae3c-5ed843cfda20/83e09dac-907c-4257-8214-2ed7a2c37859.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-24/93706cc2-93e6-4196-8f8b-f0f805749571/63aa081c-63e2-4137-a3ea-b493919dbb1a.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-17/b5840fb2-aadf-4b59-846d-998a0b471786/0c684b85-4ce8-4046-933a-88ed97f91642.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-24/f7d05ec4-af31-4c22-930d-28c7aa66781e/facb205e-a5c7-47cf-b028-8e9af07325c5.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-24/8be62e55-b3c9-455b-90fa-403966bf8a1b/f4150c19-2566-4975-bafe-fb73e22e59d7.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-17/e531c532-cd01-4ae5-aa83-9c3fd3187128/67c4eb88-b77a-4b71-9de0-79b9ec24cd8b.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-24/8be62e55-b3c9-455b-90fa-403966bf8a1b/b85f6ac2-be78-4215-b45d-b326d7d9da4e.flac
url: h

url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-04/881a897a-7f3f-48c3-8046-146a4c85d87e/42c6cfb1-44f2-46fd-aba7-224d4a3367e8.wav
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2021-01-14/019F3IH86OFULB386S3182LAES00013V/94dbec39-51d9-4426-848f-ec05679714ec.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2021-01-11/019F3IH86OFULB386S3182LAES0000KM/e4c02996-95d0-49d4-a913-7b0578ec8e15.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-11-23/a5147aa4-68ef-4424-910f-0838484badec/c6a97cdb-28fd-49cf-a4f2-bf2401c828d9.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-12-04/2bae99e9-de87-4c9a-9290-c58bd26e599f/74a4627e-68d0-4196-812b-99afc4655818.wav
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2021-01-14/019F3IH86OFULB386S3182LAES000119/26248c42-a4c6-40de-b24c-5fa6657abcae.flac
url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2021-01-14/019F3IH86OFULB386S3182LAES000119/9bb95885-36c9-45fb-8fdc-3a716d7df426.flac
url: https://hermes-cca.

url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-09-04%2Fb0600434-e0ac-4811-b4c9-c0b710aec82d%2F334f7257-f2d1-4be8-8232-18aa68f1fb2d.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F0d8bf6e3-ae6c-4e29-aa90-215c89186300%2F1d3733dc-cc1d-41f6-acf9-27757ede7e6e.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-25%2F02d99193-af3a-4807-8851-155ee1162042%2F8389a818-d4f8-47e8-b940-9ce4037e93d1.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-09-04%2Fa1f2d847-b892-4337-863b-b0b7181b0e49%2F10f05869-5b2c-4364-836b-a298f0d2cfe0.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-24%2F4ff8abec-1f84-4839-baa1-a81aae5af8a1%2F226f5075-d31f-4aca-a9a5-77f0801a181d.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-24%2Fc6be57c4-693f-4011-b5c0-27d2714222a9%2F76380e04-6c3e-49c7-bcad-0779398de2d9.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2020-08-24%2F4ff8abec-1f84-4839-baa1-a81aae5af8a1%2Fcbaded

url: https://hermes-cca.s3.ap-south-1.amazonaws.com/2020-10-08/1fe5314a-340a-47fc-ae13-9e21f319b556/2f5b5b78-c8ab-4de2-8989-265add4619e4.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003N3%2Fecb3aa08-ae67-4580-8dd6-3d46d4b68bd4.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003N3%2Fd261b9c3-757d-474e-857a-0046623a681e.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003LV%2Fd5c681c4-d8d7-4e1b-b03f-9659cd9b8561.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003JJ%2Ff86e3bc8-85ec-404f-9bc3-1b0e50aa4214.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003IO%2Fc012c835-5bb8-4f09-ad32-d71c2c6c4a14.flac
url: https://s3.ap-south-1.amazonaws.com/hermes-cca%2F2021-02-12%2F019F3IH86OFULB386S3182LAES0003IG%2F69fbc801-796a-4ffc-9bef-bb746d9ec169

In [38]:
audios_list = os.listdir("tatasky_audios")
for file in audios_list:
    #command = "ffmpeg -i " + audios/{file} + " " + file_name + ".wav"
    file_name, ext = os.path.splitext(file)
    wav_filename = file_name + ".wav"
    os.system(f"ffmpeg -i tatasky_audios/{file} tatasky_audios/{wav_filename}")  
    #os.system(f"ffmpeg -i tatasky_audios/{wav_filename} -acodec pcm_s16le -ac 1 -ar 8k tatasky_wav_audios/{file}")    

In [6]:
audios_list = os.listdir("tatasky_audios")
for file in audios_list:
    #command = "ffmpeg -i " + audios/{file} + " " + file_name + ".wav"  
    os.system(f"ffmpeg -i tatasky_audios/{file} -acodec pcm_s16le -ac 1 -ar 8k tatasky_wav_audios/{file}")    